#  IA para Redes de Suministro 

👤 **Autor:** John Leonardo Vargas Mesa  
🔗 [LinkedIn](https://www.linkedin.com/in/leonardovargas/) | [GitHub](https://github.com/LeStark)  

## 📂 Repositorio en GitHub  
- 📓 **Notebooks:** [Acceder aquí](https://github.com/LeStark/Cursos/tree/main/02%20-%20IA4SC)  
- 📑 **Data sets:** [Acceder aquí](https://github.com/LeStark/Cursos/tree/main/00%20-%20Data/02%20-%20SC)  
---

# 📘 Notebook 4 – Introducción a Computer Vision en Cadenas de Suministro  

En este notebook exploraremos los fundamentos de la **Visión por Computador (Computer Vision)**, un área de la inteligencia artificial enfocada en que las máquinas puedan interpretar y comprender información visual a partir de imágenes o videos.  

La visión por computador tiene múltiples aplicaciones en el ámbito de la cadena de suministro, como la **inspección automática de calidad**, el **reconocimiento de objetos en bodegas**, el **conteo de productos** o la **detección de incidentes en transporte y logística**.  

### Estructura del Dataset  

Para esta introducción utilizaremos imágenes que contienen distintos objetos de interés (ejemplo: personas, vehículos, elementos de transporte o carga). Estas imágenes serán empleadas como base para:  

- Comprender cómo se representan y procesan las imágenes en forma de matrices de pixeles.  
- Aplicar transformaciones básicas para resaltar características visuales.  
- Utilizar un modelo preentrenado (ej. **YOLOv8**) para realizar detección de objetos en tiempo real.  

### 🎯 Objetivos del Notebook  
- Comprender cómo se representan digitalmente las imágenes y cómo manipularlas con Python.  
- Aplicar operaciones básicas de procesamiento de imágenes (conversión a escala de grises, redimensionamiento, detección de bordes).  
- Introducir el uso de modelos de **detección de objetos preentrenados** para identificar elementos relevantes en imágenes.  
- Analizar la aplicabilidad de Computer Vision en problemas reales de **logística y cadena de suministro**.  

### 🛠️ Herramientas a utilizar  
- **OpenCV (cv2)**: lectura, visualización y transformaciones de imágenes.  
- **numpy**: manipulación matricial de datos de imagen.  
- **matplotlib**: visualización de imágenes y resultados.  
- **Ultralytics YOLOv8**: modelo preentrenado para detección de objetos.  

✅ Al finalizar este notebook, comprenderás los conceptos básicos de **Computer Vision** y cómo aplicar técnicas de procesamiento de imágenes junto con modelos de detección de objetos para **automatizar tareas dentro de la cadena de suministro**.  


In [9]:
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image

### Lectura y visualización de imágenes con OpenCV

En esta sección cargamos y mostramos una imagen local utilizando **OpenCV**:

1. **Ruta absoluta de la imagen**: se define el path completo hacia el archivo.  
2. **Lectura de la imagen**: `cv2.imread()` abre la imagen en formato **BGR** (Blue-Green-Red).  
3. **Conversión de color**: OpenCV usa BGR por defecto, pero para visualizar en librerías como *matplotlib* se convierte a **RGB** con `cv2.cvtColor()`.  
4. **Visualización**: `cv2.imshow()` abre una ventana emergente mostrando la imagen.  
5. **Interacción**: `cv2.waitKey(0)` mantiene la ventana activa hasta que el usuario presione una tecla.  
6. **Liberar recursos**: `cv2.destroyAllWindows()` cierra todas las ventanas abiertas de OpenCV.  

⚠️ En **Jupyter Notebook**, `cv2.imshow()` puede no funcionar correctamente porque abre ventanas externas. En ese caso, es mejor usar `matplotlib.pyplot.imshow()` para visualizar la imagen directamente en la celda.

In [11]:

# =====================================================
# Ejemplo de lectura y visualización de imágenes con OpenCV
# =====================================================

# Ruta absoluta de la imagen
# Nota: el prefijo 'r' delante de la cadena convierte la ruta en "raw string"
# lo cual evita problemas con las barras invertidas en Windows.
img_path = r"C:\Users\jlvar\Dropbox\Trabajo\Clases\Github_Repo\Cursos\00 - Data\00 - Basicos\ciclorruta-avenida-boyaca_0.jpeg"

# Leer la imagen con OpenCV (por defecto en formato BGR)
img = cv2.imread(img_path)

# Convertir de BGR (formato interno de OpenCV) a RGB
# Esto es necesario para mostrar correctamente la imagen en librerías como matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img_pil = Image.fromarray(img_rgb)  # convierte de NumPy a PIL
img_pil.show()

# Mostrar la imagen en una ventana emergente con OpenCV
# cv2.imshow("Imagen", img)

# Mantener la ventana abierta hasta que se presione una tecla
# cv2.waitKey(0)

# Cerrar todas las ventanas abiertas de OpenCV
# cv2.destroyAllWindows()


### Uso de un modelo preentrenado en COCO

En este paso utilizamos **YOLOv8**, un modelo de *detección de objetos* que ya fue **preentrenado en el dataset COCO**.  

- **¿Qué es COCO?**  
  Es un dataset con más de **300.000 imágenes** y **80 categorías de objetos**, incluyendo personas, vehículos, animales, mobiliario urbano, etc.  
  Al estar entrenado sobre tanta variedad, el modelo puede reconocer objetos comunes en diferentes contextos sin necesidad de que nosotros lo entrenemos desde cero.  

- **Ventaja de usar un modelo preentrenado:**  
  Entrenar una red neuronal de este tipo desde cero requiere millones de imágenes y recursos computacionales muy altos.  
  Al usar un modelo ya entrenado, **aprovechamos su conocimiento previo** y podemos:  
  1. Usarlo directamente para detectar objetos en nuestras imágenes (*inferencia*).  
  2. Adaptarlo a un nuevo dominio (*transfer learning / fine-tuning*) con un dataset más pequeño y específico.  

- **En esta celda:**  
  1. Cargamos el modelo `yolov8n.pt` (versión ligera de YOLOv8).  
  2. Lo aplicamos sobre nuestra imagen de prueba.  
  3. El resultado es una lista de objetos detectados con sus clases y niveles de confianza.  

Esto nos permite comprobar de manera inmediata el poder de las **redes neuronales profundas entrenadas en grandes datasets** y sentar las bases para aplicarlas luego en problemas específicos de la cadena de suministro.


In [12]:

# =====================================================
# Inferencia con un modelo YOLOv8 preentrenado en COCO
# =====================================================

# Cargar modelo YOLOv8 (versión 'n' = nano, más ligera y rápida)
# El modelo viene preentrenado en el dataset COCO, que contiene 80 clases de objetos comunes
# como personas, vehículos, animales, señales de tránsito, etc.
model = YOLO("yolov8n.pt")

# Realizar la inferencia sobre la imagen cargada previamente
# El modelo detectará los objetos de la imagen basándose en el conocimiento adquirido
# durante su entrenamiento en COCO
results = model(img_path)



image 1/1 C:\Users\jlvar\Dropbox\Trabajo\Clases\Github_Repo\Cursos\00 - Data\00 - Basicos\ciclorruta-avenida-boyaca_0.jpeg: 384x640 3 persons, 4 bicycles, 2 buss, 1 truck, 1 backpack, 45.9ms
Speed: 1.4ms preprocess, 45.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


In [13]:
# Mostrar predicciones
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [14]:
res = results[0]
res.show()  # abre la imagen con las detecciones

In [15]:
from collections import Counter

res = results[0]

# Crear un contador de clases
conteo = Counter(int(box.cls[0]) for box in res.boxes)

# Mostrar resultados
for cls_id, cantidad in conteo.items():
    nombre = res.names[cls_id]
    print(f"{nombre}: {cantidad}")

bicycle: 4
person: 3
bus: 2
truck: 1
backpack: 1


### Detección en tiempo real con YOLOv8 y cámara web

En esta celda implementamos un sistema de **detección de objetos en tiempo real** utilizando un modelo **YOLOv8 preentrenado** y la cámara web del equipo.

1. **Carga del modelo preentrenado**  
   Se utiliza la versión `yolov8`, un modelo pequeño y rápido, entrenado en el dataset **COCO (80 clases)**.  
   La ventaja de usar un modelo preentrenado es que ya reconoce objetos comunes (personas, vehículos, animales, etc.) sin necesidad de entrenarlo desde cero.  

2. **Inicialización de la cámara web**  
   Con `cv2.VideoCapture(0)` se abre la primera cámara disponible.  

3. **Bucle de captura e inferencia**  
   - En cada iteración se lee un frame de la cámara.  
   - El frame se pasa al modelo YOLOv5 para realizar la detección.  
   - Con `results[0].plot()` se obtiene una versión anotada de la imagen con las cajas y etiquetas de los objetos detectados.  
   - Se muestra el resultado en una ventana llamada *"YOLOv5 - Detección en tiempo real"*.  

4. **Interacción con el usuario**  
   El bucle se mantiene hasta que el usuario presione la tecla **q**.  

5. **Liberación de recursos**  
   Finalmente, se libera la cámara y se cierran las ventanas de OpenCV.  

 Este ejemplo muestra cómo integrar un **modelo de visión por computador de última generación** con una fuente de video en vivo, lo que abre la puerta a aplicaciones en **logística, seguridad, transporte, inspección de calidad** y muchos otros escenarios.


In [ ]:
# =====================================================
# Cargar modelo YOLOv5 preentrenado desde GitHub
# =====================================================
# "yolov5s" = modelo pequeño (small), más rápido y ligero.
# También existen variantes más grandes y precisas: "yolov5m", "yolov5l", "yolov5x".

# =====================================================
# Inicializar la cámara web
# =====================================================
# 0 -> primera cámara disponible (cámara por defecto)
cap = cv2.VideoCapture(0)

while True:
    # Leer un frame de la cámara
    ret, frame = cap.read()
    if not ret:
        break  # Si no hay frame, salimos del bucle

    # =====================================================
    # Realizar inferencia con YOLOv5
    # =====================================================
    results = model(frame)  # Corre el modelo sobre el frame
    annotated_frame = results[0].plot()
    # Mostrar el frame con las detecciones en una ventana
    cv2.imshow('YOLOv8 - Detección en tiempo real', annotated_frame)

    # =====================================================
    # Tecla de salida
    # =====================================================
    # Si se presiona la tecla 'q', se rompe el bucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# =====================================================
# Liberar recursos
# =====================================================
cap.release()           # Libera la cámara
cv2.destroyAllWindows() # Cierra todas las ventanas de OpenCV


### Transfer Learning y Fine-tuning con YOLOv8

En las celdas a continuación aplicaremos **transfer learning** y **fine-tuning** para que el modelo YOLOv8, inicialmente preentrenado en COCO, aprenda a **identificar cajas de almacenamiento** dentro de nuestro dataset específico.  
Este proceso nos permitirá adaptar un modelo generalista a un problema concreto en el contexto de la cadena de suministro.


In [16]:
# =====================================================
# Inferencia con un modelo YOLOv8 preentrenado en COCO
# =====================================================

# Cargar el modelo YOLOv8 (versión 'n' = nano, ligera y rápida)
# Este modelo ya está entrenado en el dataset COCO, que contiene 80 clases
# de objetos comunes (personas, vehículos, animales, mobiliario, etc.)
model_box = YOLO("yolov8n.pt")

# Definir la ruta de la imagen de prueba
# En este caso, una fotografía que contiene una caja (ejemplo práctico)
ejemplo_caja = r"C:\Users\jlvar\Dropbox\Trabajo\Clases\Github_Repo\Cursos\00 - Data\00 - Basicos\ejemplo_caja.jpeg"

# Realizar la inferencia sobre la imagen
# El modelo detectará los objetos presentes y devolverá coordenadas,
# clases y niveles de confianza para cada detección
results = model_box(ejemplo_caja)

# Mostrar la imagen con las detecciones visualizadas
# (abre una ventana externa con las cajas dibujadas alrededor de los objetos detectados)
results[0].show()



image 1/1 C:\Users\jlvar\Dropbox\Trabajo\Clases\Github_Repo\Cursos\00 - Data\00 - Basicos\ejemplo_caja.jpeg: 640x448 1 person, 1 cup, 1 bowl, 67.5ms
Speed: 3.5ms preprocess, 67.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 448)


### Entrenamiento del modelo con dataset personalizado

En esta celda entrenamos **YOLOv8** aplicando **transfer learning** sobre un dataset específico de cajas de almacenamiento.  

1. **Archivo `data.yaml`**  
   Define la estructura del dataset, incluyendo:
   - Rutas a las carpetas `train/`, `val/` y `test/`.  
   - Número de clases (`nc`).  
   - Nombres de las clases.  

2. **Parámetros de entrenamiento**  
   - `epochs=10`: el modelo se entrena durante 10 ciclos completos sobre el dataset.  
   - `imgsz=640`: todas las imágenes se redimensionan a 640×640 píxeles.  
   - `batch=16`: se procesan 16 imágenes por iteración.  
   - `name="yolov8_box_inventory"`: los resultados se almacenan en `runs/detect/yolov8_box_inventory`.  
   - `fraction=0.05`: se utiliza solo el **5% del dataset** para pruebas rápidas (útil para validar que todo funciona antes de entrenar con el 100%).  

Este proceso nos permite adaptar un modelo preentrenado en **COCO** a un caso de uso específico en la **cadena de suministro**, enseñándole a reconocer cajas de almacenamiento con un dataset reducido.


In [ ]:
# =====================================================
# Entrenamiento de YOLOv8 con dataset personalizado
# =====================================================

# Ruta al archivo YAML de configuración del dataset
# Este archivo define las rutas de entrenamiento, validación y prueba,
# así como el número de clases y sus nombres.
data_path = r"D:\Data Sets CV\CV-Inventory.v1-ia4sc-box-count-example.yolov8\data.yaml"

# Entrenar el modelo YOLOv8 usando transferencia de aprendizaje
# En este caso, se parte de un modelo preentrenado y se adapta
# para detectar "cajas de almacenamiento" en nuestro dataset específico.
model_box.train(
    data=data_path,        # Dataset personalizado (definido en data.yaml)
    epochs=10,             # Número de épocas de entrenamiento
    imgsz=640,             # Tamaño de las imágenes (redimensionamiento a 640x640)
    batch=16,              # Tamaño del lote de entrenamiento
    name="yolov8_box_inventory",  # Carpeta donde se guardarán resultados y pesos
    fraction=0.05          # Usar solo el 5% del dataset (ideal para pruebas rápidas)
)

Ultralytics 8.3.205  Python-3.12.7 torch-2.8.0+cpu CPU (Intel Core(TM) Ultra 7 155H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Data Sets CV\CV-Inventory.v1-ia4sc-box-count-example.yolov8\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.05, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_box_inventory9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overla

In [ ]:
model_box.val()


In [ ]:
# Cargar la mejor versión entrenada
best_model_box = YOLO("runs/detect/yolov8_box_inventory/weights/best.pt")
best_model_box.save("best_box_model.pt")


In [ ]:
# Inferencia en una imagen de prueba
results = best_model_box(ejemplo_caja)
results[0].show()